In [1]:
import pandas as pd
import datetime

In [3]:
#df_users = pd.read_csv("/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4012/Data/combined_twitter_data_with_tweets_corpus.csv")
df_users = pd.read_csv("/Users/wenyin/Desktop/BT4012/Data/combined_twitter_data_with_tweets_corpus.csv")

In [9]:
# get tweets df
#base = "/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4012/"
base = '/Users/wenyin/Desktop/BT4012/'
filenames_tweets = [
    "Data/all tweets 2017/tweets_fake_followers.csv",
    "Data/all tweets 2017/tweets_genuine_accounts.csv",
    "Data/all tweets 2017/tweets_social_spambots_1.csv",
    "Data/all tweets 2017/tweets_social_spambots_2.csv",
    "Data/all tweets 2017/tweets_social_spambots_3.csv",
    "Data/all tweets 2017/tweets_traditional_spambots_1.csv",

    "Data/tweets 2015/tweets_E13.csv",
    "Data/tweets 2015/tweets_FSF.csv",
    "Data/tweets 2015/tweets_INT.csv",
    "Data/tweets 2015/tweets_TFP.csv",
    "Data/tweets 2015/tweets_TWT.csv"
]
filenames_tweets = map(lambda x: base+x, filenames_tweets)

for i,fn in enumerate(filenames_tweets):
    if i == 0:
        df_tweets = pd.read_csv(fn, encoding='ISO-8859-1')
    else:
        df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)


In [10]:
df_tweets = df_tweets.dropna(subset = ["user_id"])  
df_tweets["user_id"] = df_tweets["user_id"].apply(int)

## Date Formatting

In [11]:
# takes around 10 min to run
df_tweets['created_at_formatted'] = pd.to_datetime(df_tweets['timestamp'], infer_datetime_format=True, errors='coerce')

In [ ]:
df_tweets['created_at_date'] = df_tweets['created_at_formatted'].apply(lambda x: x.date())

## User Tweet Frequency

In [13]:
# user tweet frequency = total number of tweets / number of user active days 
# shows how often the user tweets among the days that a user tweets at least once. User activity is defined by whether the user tweets in a given day
# 1 = user tweets only once per active day 
# >1 = user tweets more than once a day on average, in the days that the user is active 

df_tweets_per_day = df_tweets.groupby(by=["user_id"]).agg(tweet_count=('text', 'count'),
                                                          date_count=('created_at_date', lambda x: x.nunique()))

dict_tweets_average = {user_id: df_tweets_per_day.loc[user_id]['tweet_count'] / df_tweets_per_day.loc[user_id]['date_count'] for user_id in df_tweets_per_day.index}

In [14]:
#create new column for user tweet frequency 
df_users['tweet_frequency'] = df_users['id'].map(dict_tweets_average)

## Tags and Mentions

In [15]:
# average number of tags per post = total number of tags used per tweet 
# average number of mentions per post = total number of mentions per tweet 

df_tweets['text'] = df_tweets['text'].apply(str) #convert all text to string
df_tweets['number_of_tags'] = df_tweets['text'].apply(lambda x: x.count("#"))
df_tweets['number_of_mentions'] = df_tweets['text'].apply(lambda x: x.count("@"))

In [16]:
tags_dict = df_tweets.groupby(by=["user_id"])['number_of_tags'].sum().to_dict()
mentions_dict = df_tweets.groupby(by=["user_id"])['number_of_mentions'].sum().to_dict() 

#create new column for number of tags
df_users['number_of_tags'] = df_users['id'].map(tags_dict)
#create new column for number of mentions
df_users['number_of_mentions'] = df_users['id'].map(mentions_dict)


## User tweets on weekdays and weekends

In [17]:
# return 0 if weekend, 1 if weekday 
def is_weekday(dt):
    return 0 if dt.weekday() > 4 else 1

# return day of week 
def get_weekday(dt):
    return dt.weekday()

In [ ]:
def get_weekend_weekday_frequency(df_tweets, df_users):
    df_tweets['weekday'] = df_tweets['created_at_formatted'].apply(lambda x: is_weekday(x))
    df_tweets_weekday_weekend = df_tweets.groupby(by=["user_id", "weekday"]).agg(tweet_count=('text', 'count'),
                                                          date_count=('created_at_date', lambda x: x.nunique()))
    dict_tweets_weekend = {user_id: df_tweets_weekday_weekend.loc[(user_id, weekday)]['tweet_count'] / df_tweets_weekday_weekend.loc[(user_id, weekday)]['date_count'] for (user_id, weekday) in df_tweets_weekday_weekend.index if weekday == 0}
    df_users['tweet_weekend_frequency'] = df_users['id'].map(dict_tweets_weekend)        
    dict_tweets_weekday = {user_id: df_tweets_weekday_weekend.loc[(user_id, weekday)]['tweet_count'] / df_tweets_weekday_weekend.loc[(user_id, weekday)]['date_count'] for (user_id, weekday) in df_tweets_weekday_weekend.index if weekday == 1}
    df_users['tweet_weekday_frequency'] = df_users['id'].map(dict_tweets_weekday)
    return df_users             

In [42]:
df_users.head()

,Unnamed: 0.1,Unnamed: 0,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,...,description,created_at,updated,account_type,tweets_list,tweet_frequency,number_of_tags,number_of_mentions,tweet_weekend_frequency,tweet_weekday_frequency
0,0,1,22903,effeffe,effeffe,164,132,194,12,4,...,L'uomo ha creato dio a sua immagine e somiglia...,Sun Nov 26 15:19:32 +0000 2006,14/2/2015 11:32,real,['@TheFakeProject cerca followers reali!!! #Im...,1.842697,28.0,126.0,1.684211,1.885714
1,1,3,286543,Alessio Bragadini,abragad,6892,930,535,478,28,...,Web and social media developer from Italy,Wed Dec 27 14:55:17 +0000 2006,14/2/2015 11:32,real,"[""RT @hotdogsladies: The real problem with ema...",4.438070,1051.0,2441.0,4.652968,4.352190
2,2,4,438023,fullcaffeine,fullcaffeine,2885,173,444,41,2,...,NaN,Tue Jan 02 09:01:50 +0000 2007,14/2/2015 11:32,real,['Amare il #giornalismo : @Internazionale @la_...,5.971014,128.0,368.0,5.000000,6.306407
3,3,5,586003,Maurizio Tesconi,myself2048,216,97,234,2,0,...,CNR Researcher. Web developer. Social media ge...,Fri Jan 05 16:20:42 +0000 2007,14/2/2015 11:32,real,"['RT @TheFakeProject: Dear followers, phase 2 ...",1.745763,160.0,240.0,1.607143,1.788889
4,4,6,628563,Massimo Moretti,MaxMoretti,505,154,314,0,3,...,The truth is out there!,Fri Jan 12 11:06:17 +0000 2007,14/2/2015 11:32,real,['@Swype when the Italian dictionary will be ...,2.193833,51.0,106.0,2.311688,2.133333
